In [1]:
import warnings; warnings.filterwarnings("ignore")

In [2]:
from IPython.core.display import HTML; 
display(HTML("<style>.p-Widget.jp-OutputPrompt.jp-OutputArea-prompt:empty {padding: 0; border: 0;}</style>"));

In [3]:
import sys, os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from glob import glob

In [4]:
def process_result_csvs(asset_dir,tag=None):
    asset_dflist = []
    assets = glob(asset_dir + '/*.csv')
    for asset in tqdm(assets, leave = False):
        asset_subdir = asset.split('/')[-2]
        asset_name = asset.split('/')[-1].split('.')[0]
        asset_parse = asset_name.split('_')
        df = pd.read_csv(asset)
        df['model'] = '_'.join(asset_subdir.split('_')[:-1])
        df['training'] = asset_subdir.split('_')[-1]
        if 'feature_map' in list(df.columns):
            df = df.rename(columns={'feature_map': 'model_layer'})
        df['brain_area'] = asset_parse[-2]
        df['brain_layer'] = asset_parse[-1]
        df = df[['brain_area', 'brain_layer', 'neuron', 'model', 'training', 'model_layer', 'score']]
        asset_dflist.append(df)
    asset_df = pd.concat(asset_dflist)
    return asset_df

In [5]:
process_result_csvs('srp_ridge_results/alexnet_imagenet')

,brain_area,brain_layer,neuron,model,training,model_layer,score
0,VISal,layer4,0,alexnet,imagenet,Linear-2,0.072238
1,VISal,layer4,1,alexnet,imagenet,Linear-2,0.037682
2,VISal,layer4,2,alexnet,imagenet,Linear-2,0.052738
3,VISal,layer4,3,alexnet,imagenet,Linear-2,0.011697
4,VISal,layer4,4,alexnet,imagenet,Linear-2,0.027378
...,...,...,...,...,...,...,...
24,VISrl,layer4,24,alexnet,imagenet,ReLU-4,0.103348
25,VISrl,layer4,25,alexnet,imagenet,ReLU-4,0.037463
26,VISrl,layer4,26,alexnet,imagenet,ReLU-4,0.203415
27,VISrl,layer4,27,alexnet,imagenet,ReLU-4,0.109536


### SRP Ridge Results Processing

#### Cleaning, Missing Data Removal, Deduplication

In [6]:
source_dirs = ['srp_ridge_results']

In [7]:
output_file = 'srp_ridge_results.csv'
recompute = False

dflist = []
if not os.path.exists(output_file) or recompute:
    for source_dir in tqdm(source_dirs):
        asset_dirs = glob(source_dir + '/*')
        for asset_dir in tqdm(asset_dirs):
            dflist.append(process_result_csvs(asset_dir))

    scoresheet = pd.concat(dflist)
    scoresheet.to_csv(output_file, index = None)

scoresheet = pd.read_csv(output_file)

In [8]:
scoresheet.groupby(['training']).size().reset_index().rename(columns={0:'count'})

,training,count
0,deepcluster,632478
1,imagenet,13512030
2,random,13487388
3,taskonomy,30605364


In [9]:
cell_data = pd.read_csv('cell_data_merge.csv')
problem_cells = cell_data[(cell_data['p_run_mod_ns'] < 0.05)]['cell_specimen_id']
reliable_cells = (pd.read_csv('brain_responses_reliable_cell_ids.csv')
                  .rename(columns={'area': 'brain_area', 'layer': 'brain_layer'}))
all_target_cells = reliable_cells[~reliable_cells['cell_specimen_id'].isin(problem_cells)]

In [10]:
scoresheet = scoresheet.merge(reliable_cells, on=['brain_area', 'brain_layer', 'neuron'])
scoresheet = scoresheet[~scoresheet['cell_specimen_id'].isin(problem_cells)]
scoresheet['model_string'] = scoresheet['model'] + '_' + scoresheet['training']
scoresheet['neural_site'] = scoresheet['brain_area'] + '_' + scoresheet['brain_layer']

In [11]:
scoresheet.groupby(['training']).size().reset_index().rename(columns={0:'count'})

,training,count
0,deepcluster,509047
1,imagenet,10875095
2,random,10855262
3,taskonomy,24632586


In [12]:
shared_columns = ['brain_area', 'brain_layer', 'model', 'model_layer', 'neuron', 'cell_specimen_id']
imagenet_scoresheet = (scoresheet.query("training == 'imagenet'")[shared_columns]
       .sort_values(by=shared_columns)).reset_index(drop=True)
random_scoresheet = (scoresheet.query("training == 'random'")[shared_columns]
       .sort_values(by=shared_columns)).reset_index(drop=True)

In [13]:
imagenet_scoresheet.shape[0] - random_scoresheet.shape[0]

19833

In [14]:
(imagenet_scoresheet.groupby(['model'])['cell_specimen_id'].agg('count') - 
 random_scoresheet.groupby(['model'])['cell_specimen_id'].agg('count'))

model
alexnet              0
densenet161       6611
mnasnet1_0           0
mobilenet_v2         0
resnet152            0
resnet18         13222
resnet50             0
squeezenet1_0        0
vgg16                0
vgg16_bn             0
Name: cell_specimen_id, dtype: int64

In [15]:
(imagenet_scoresheet.groupby(['model'])['cell_specimen_id'].agg('count') - 
 random_scoresheet.groupby(['model'])['cell_specimen_id'].agg('count')).sum()

19833

In [16]:
(imagenet_scoresheet.groupby(['model'])['model_layer'].nunique() -
 random_scoresheet.groupby(['model'])['model_layer'].nunique())

model
alexnet          0
densenet161      1
mnasnet1_0       0
mobilenet_v2     0
resnet152        0
resnet18         1
resnet50         0
squeezenet1_0    0
vgg16            0
vgg16_bn         0
Name: model_layer, dtype: int64

In [17]:
merge_check = imagenet_scoresheet.merge(random_scoresheet.drop_duplicates(), 
                                        on=shared_columns, how='left', indicator=True)
missing_from_merge = merge_check[merge_check._merge == 'left_only']

In [18]:
missing_from_merge[missing_from_merge._merge == 'left_only'].shape

(13222, 7)

In [19]:
missing_from_merge.groupby(['model','model_layer']).size().reset_index().rename(columns={0:'count'})

,model,model_layer,count
0,densenet161,ReLU-2,6611
1,resnet18,AvgPool2d-1,6611


In [20]:
models_with_missing_neurons = missing_from_merge['model'].to_list()
model_layers_with_missing_neurons = missing_from_merge['model_layer'].to_list()

In [21]:
missing_data = ((imagenet_scoresheet.groupby(['brain_area','brain_layer', 'model','model_layer']).size() - 
                 random_scoresheet.groupby(['brain_area','brain_layer', 'model','model_layer']).size())
                .reset_index().rename(columns={0:'count'}))

In [22]:
missing_data.query("count != 0.0").groupby(['model','model_layer']).size().reset_index()

,model,model_layer,0
0,densenet161,ReLU-2,21
1,resnet18,AvgPool2d-1,21
2,resnet18,Linear-1,21


In [23]:
na_data = scoresheet[scoresheet['score'].isna()].groupby(['model','model_layer']).size().reset_index().rename(columns={0:'count'})

In [24]:
na_data.groupby(['model','model_layer']).size().reset_index()

,model,model_layer,0
0,mnasnet1_0,Dropout-1,1
1,mnasnet1_0,Linear-1,1
2,mobilenet_v2,Dropout-1,1
3,mobilenet_v2,Linear-1,1


In [25]:
output_file = 'possibly_missing_reg_data.csv'
if not os.path.exists(output_file):
    missing_data.query("count != 0.0").to_csv(output_file, index = None)

In [26]:
rows_to_remove = (missing_data.query("count != 0.0").groupby(['model','model_layer']).size().reset_index()[['model','model_layer']].
                  append(na_data[['model','model_layer']]))
rows_to_remove['model_layer_string'] = rows_to_remove['model'] + '_' + rows_to_remove['model_layer']

In [27]:
rows_to_remove

,model,model_layer,model_layer_string
0,densenet161,ReLU-2,densenet161_ReLU-2
1,resnet18,AvgPool2d-1,resnet18_AvgPool2d-1
2,resnet18,Linear-1,resnet18_Linear-1
0,mnasnet1_0,Dropout-1,mnasnet1_0_Dropout-1
1,mnasnet1_0,Linear-1,mnasnet1_0_Linear-1
2,mobilenet_v2,Dropout-1,mobilenet_v2_Dropout-1
3,mobilenet_v2,Linear-1,mobilenet_v2_Linear-1


In [28]:
scoresheet['model_layer_string'] = scoresheet['model'] + '_' + scoresheet['model_layer']
scoresheet = scoresheet[~scoresheet['model_layer_string'].isin(rows_to_remove['model_layer_string'])]

In [29]:
scoresheet.groupby(['training']).size().reset_index().rename(columns={0:'count'})

,training,count
0,deepcluster,509047
1,imagenet,10822207
2,random,10822207
3,taskonomy,24632586


In [30]:
output_file = 'srp_ridge_results_processed.csv'
if not os.path.exists(output_file):
    scoresheet.to_csv(output_file, index = None)